In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\

### 1 加载数据集

In [2]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

W0818 20:16:48.709696  7956 deprecation.py:323] From <ipython-input-2-83231f068ae1>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0818 20:16:48.710696  7956 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0818 20:16:48.712696  7956 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instruc

Extracting MNIST_data\train-images-idx3-ubyte.gz


W0818 20:16:48.944709  7956 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0818 20:16:48.947710  7956 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0818 20:16:48.996712  7956 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecat

Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


### 2 设定批次

In [3]:
batch_size = 50
n_batch = mnist.train.num_examples // batch_size  # 总共有多少个批次

# 定义训练集占位符
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='input_x')  # 每一行表示一个样本，内容是784为的像素值
    y = tf.placeholder(tf.float32, [None, 10], name='input_y')   # 每一行表示一个label，采用杜热编码
    x_image = tf.reshape(x, [-1,28,28,1], name='to_an_image')  # 输入转换为图片的pattern
    
keep_prob = tf.placeholder(tf.float32)


# 参数概要
def variable_summary(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('平均值', mean)               # 求平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('方差', stddev)           # 标准差
        tf.summary.scalar('最大值', tf.reduce_max(var))  # 最大值
        tf.summary.scalar('最小值', tf.reduce_min(var))  # 最小值
        tf.summary.histogram('直方图', var)        # 直方图

### 3 初始化权重和偏差

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape=shape, stddev=0.1, name='weight_hahaha')
    return tf.Variable(initial)


def bias_variable(shape):  # 初始化为0.1
    initial = tf.constant(0.1, shape=shape, name='bias_hahaha')
    return tf.Variable(initial)

### 卷积层与池化层

In [5]:
def conv2d(x, w):  # 2维卷积
    # x:输入张量 shape=【批量， 图长， 图宽， 通道】，灰度的通道为1，彩色为3
    # w：filter 卷积核张量 shape=【高度， 宽度， 输入通道， 输出通道】输出通道等于卷积核的数
    # stride：[batch, height, width, channels].
    return tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME')


def max_pool_2mul2(x):
    # ksize: [1,x,y,1]窗口大小
    # stride: 步长
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


### 3 创建安神经网络（不含隐藏层）

In [6]:
with tf.name_scope('CNN'):
    # 初始化第一个卷积层的权重和偏置
    with tf.name_scope('conv_pooling'):
        w_conv1 = weight_variable([5,5,1,32])  # 5乘5的采样窗口 32个卷积核
        b_conv1 = bias_variable([32])          # 每个卷积核一个偏置
        h_conv1 = tf.nn.relu(conv2d(x_image, w_conv1) + b_conv1)  # 卷积
        h_pool1 = max_pool_2mul2(h_conv1)                         # 池化

        # 初始化第二个卷积层的权重和偏差
        w_conv2 = weight_variable([5,5,32,64])  #55的窗口，通道数为32， 卷积核64
        b_conv2 = bias_variable([64])
        h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
        h_pool2 = max_pool_2mul2(h_conv2)
    
    with tf.name_scope('NN'):
        # 定义两个全连接层
        w_fc1 = weight_variable([7*7*64, 500])  #1024 hidden unit
        b_fc1 =bias_variable([500])
        h_pool2_flat = tf.reshape(h_pool2, [-1,7*7*64])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)
        h_fc1_prob = tf.nn.dropout(h_fc1, keep_prob=keep_prob)

        w_fc2 = weight_variable([500, 10])  #10 hidden unit
        b_fc2 =bias_variable([10])
        prediction = tf.nn.softmax(tf.matmul(h_fc1_prob, w_fc2) + b_fc2)

W0818 20:16:50.112776  7956 deprecation.py:506] From <ipython-input-6-2a01baf4396c>:21: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### 4 优化器

In [7]:
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    tf.summary.scalar('loss', loss)
    
with tf.name_scope('train_step'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

W0818 20:16:50.268785  7956 deprecation.py:323] From <ipython-input-7-5d028351f30a>:2: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### 5 获取预测结果并计算准确率

In [8]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.arg_max(prediction, 1), tf.arg_max(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

W0818 20:16:50.477797  7956 deprecation.py:323] From <ipython-input-8-b86b710cb5ec>:2: arg_max (from tensorflow.python.ops.gen_math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.math.argmax` instead


### 6 启动会话

In [9]:
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('G:/tensorflow/logs/',sess.graph)
    for epoch in range(21):
        time1 = time.time()
        for batch in range(n_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, summary = sess.run([train_step, merged], feed_dict={x:batch_x, y:batch_y, keep_prob:1.0})  
        writer.add_summary(summary, epoch)
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})  # 用测试集来预测
        print("第{0}圈  准确率是： {1}, 耗时: {2} 秒 ".format(epoch, acc, time.time()-time1))

第0圈  准确率是： 0.8518000245094299, 耗时: 72.90117001533508 秒 
第1圈  准确率是： 0.8837000131607056, 耗时: 71.85310983657837 秒 
第2圈  准确率是： 0.9797000288963318, 耗时: 72.58815169334412 秒 
第3圈  准确率是： 0.9800000190734863, 耗时: 73.41619920730591 秒 
第4圈  准确率是： 0.986299991607666, 耗时: 73.58420872688293 秒 
第5圈  准确率是： 0.9865000247955322, 耗时: 73.67821383476257 秒 
第6圈  准确率是： 0.9872999787330627, 耗时: 74.14224076271057 秒 
第7圈  准确率是： 0.9890000224113464, 耗时: 73.54920673370361 秒 
第8圈  准确率是： 0.9887999892234802, 耗时: 73.95923018455505 秒 
第9圈  准确率是： 0.986299991607666, 耗时: 74.44425821304321 秒 
第10圈  准确率是： 0.9890000224113464, 耗时: 77.4294285774231 秒 
第11圈  准确率是： 0.9904000163078308, 耗时: 80.97963166236877 秒 
第12圈  准确率是： 0.9889000058174133, 耗时: 85.00786232948303 秒 
第13圈  准确率是： 0.9908999800682068, 耗时: 73.90322709083557 秒 
第14圈  准确率是： 0.9908999800682068, 耗时: 73.88722610473633 秒 
第15圈  准确率是： 0.9905999898910522, 耗时: 75.53932070732117 秒 
第16圈  准确率是： 0.9909999966621399, 耗时: 109.05223751068115 秒 
第17圈  准确率是： 0.9900000095367432, 耗时: 90.6621